In [9]:
# Stocks!
# Need this form:
# List of these lists:
# [all macro eco features and derived features, previous month all macro features] --> label:price

#https://www.analyticsvidhya.com/blog/2016/02/time-series-forecasting-codes-python/

'''
#weighted moving average
import pandas, numpy
ewma = pandas.stats.moments.ewma
EMOV_n = ewma( ys, com=2 )
#Here, com is a parameter that you can read about here. Then you can combine EMOV_n to Xs, using something like:

Xs = numpy.vstack((Xs,EMOV_n))
#And then you can look at various linear models, here, and do something like:

from sklearn import linear_model
clf = linear_model.LinearRegression()
clf.fit ( Xs, ys )
print clf.coef_
'''
import pandas as pd
import numpy as np

indicators = pd.read_csv('Indicators.csv').fillna(0)
prices = pd.read_csv('stocks-us-adjClose.csv').fillna(0)


print(indicators.columns)
print(prices.columns)
print(prices.head(1))
print(prices.shape)

Index(['DATE', 'PAYEMS', 'PAYNSA', 'AHETPI', 'USCONS', 'USCONS_NSA', 'MANEMP',
       'MANEMP_NSA', 'USTRADE', 'HOUST', 'UNRATE', 'UNRATENSA', 'EMRATIO',
       'UEMPMEAN', 'UEMPMED', 'U2RATE', 'U1RATE', 'CPIAUCSL', 'UMCSENT',
       'RRSFS', 'RSXFS', 'USSLIND', 'SPCS20RSA', 'SPCS2-RNSA', 'KCFSI',
       'DGORDER', 'IPMAN'],
      dtype='object')
Index(['compiled from Yahoo! Finance data by Matt Borthwick', 'ED', 'DD',
       'CVX', 'FL', 'CAT', 'IP', 'SJW', 'F', 'LLY',
       ...
       'EXTN', 'VYGR', 'ACG', 'MIME', 'TCRZ', 'MCX', 'EDIT', 'LMHA', 'UA',
       'BTU'],
      dtype='object', length=711)
  compiled from Yahoo! Finance data by Matt Borthwick        ED       DD  \
0                                         1970-01-02   0.307997  0.00048   

        CVX        FL       CAT        IP  SJW    F  LLY ...   EXTN  VYGR  \
0  0.582503  1.857836  1.475225  1.808415  0.0  0.0  0.0 ...    0.0   0.0   

   ACG  MIME  TCRZ  MCX  EDIT  LMHA   UA  BTU  
0  0.0   0.0   0.0  0.0   0.0   0.